In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier

from collections import Counter

In [141]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
PassengerId = df_test['PassengerId']
df_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [142]:
def detect_outliers(df,n,features):
    outlier_indices = []
    for col in features:
        Q1 = np.percentile(df[col], 25)
        Q3 = np.percentile(df[col],75)
        IQR = Q3 - Q1
        outlier_step = 1.5 * IQR
        outlier_list_col = df[(df[col] < Q1 - outlier_step) | (df[col] > Q3 + outlier_step )].index
        outlier_indices.extend(outlier_list_col)
    outlier_indices = Counter(outlier_indices)
    multiple_outliers = list( k for k, v in outlier_indices.items() if v > n )
    return multiple_outliers   

to_drop = detect_outliers(df_train,2,["Age","SibSp","Parch","Fare"])

C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\function_base.py:4291: RuntimeWarning: Invalid value encountered in percentile
  interpolation=interpolation)


In [143]:
df_train = df_train.drop(to_drop, axis=0).reset_index(drop=True)

In [144]:
df = pd.concat(objs=[df_train, df_test], axis=0).reset_index(drop=True)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  """Entry point for launching an IPython kernel.


In [145]:
df.isnull().sum()

Age             256
Cabin          1007
Embarked          2
Fare              1
Name              0
Parch             0
PassengerId       0
Pclass            0
Sex               0
SibSp             0
Survived        418
Ticket            0
dtype: int64

In [146]:
df['Fare'] = df['Fare'].fillna(df_test['Fare'].median())
df['Fare'] = df['Fare'].map(lambda i: np.log(i) if i>0 else 0)

In [147]:
df['Embarked'] = df['Embarked'].fillna('S')

In [148]:
sexEncoder = LabelEncoder()
df["Sex"] = sexEncoder.fit_transform(df["Sex"])

In [149]:
# filling missing values of Age
age_nan = list(df['Age'][df['Age'].isnull()].index)

for i in age_nan:
    median = df['Age'].median()
    df['Age'].iloc[i] = median

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [150]:
df['Name'][:3]

0                              Braund, Mr. Owen Harris
1    Cumings, Mrs. John Bradley (Florence Briggs Th...
2                               Heikkinen, Miss. Laina
Name: Name, dtype: object

In [151]:
df['Title'] = [i.split(', ')[1].split('.')[0] for i in df['Name']]

In [152]:
df['Title'].unique()

array(['Mr', 'Mrs', 'Miss', 'Master', 'Don', 'Rev', 'Dr', 'Mme', 'Ms',
       'Major', 'Lady', 'Sir', 'Mlle', 'Col', 'Capt', 'the Countess',
       'Jonkheer', 'Dona'], dtype=object)

In [153]:
df['Title'] = df['Title'].replace(['Miss','Mme','Mlle','Mrs'], 'Ms')
df['Title'] = df["Title"].replace(['Lady', 'the Countess','Countess','Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
titleEncoder = LabelEncoder()
df['Title'] = titleEncoder.fit_transform(df['Title'])

In [154]:
df = df.drop(columns=['Name', 'PassengerId'])

In [155]:
embEncoder = LabelEncoder()
df['Embarked'] = embEncoder.fit_transform(df['Embarked'])

In [125]:
# df = pd.get_dummies(df, columns=['Pclass', 'Title', 'Embarked'])

In [156]:
df.head(10)

,Age,Cabin,Embarked,Fare,Parch,Pclass,Sex,SibSp,Survived,Ticket,Title
0,22.0,NaN,2,1.981001,0,3,1,1,0.0,A/5 21171,1
1,38.0,C85,0,4.266662,0,1,0,1,1.0,PC 17599,2
2,26.0,NaN,2,2.070022,0,3,0,0,1.0,STON/O2. 3101282,2
3,35.0,C123,2,3.972177,0,1,0,1,1.0,113803,2
4,35.0,NaN,2,2.085672,0,3,1,0,0.0,373450,1
5,28.0,NaN,1,2.135148,0,3,1,0,0.0,330877,1
6,54.0,E46,2,3.948596,0,1,1,0,0.0,17463,1
7,2.0,NaN,2,3.048088,1,3,1,3,0.0,349909,0
8,27.0,NaN,2,2.409941,2,3,0,0,1.0,347742,2
9,14.0,NaN,0,3.403555,0,2,0,1,1.0,237736,2


In [157]:
df = df.drop(columns=['Cabin', 'Ticket'])

In [158]:
train = df[:len(df_train)]
test = df[:len(df_test)]
X_test = test.drop(columns=['Survived'])

In [159]:
X_train = train.drop(columns=['Survived'])
Y_train = train['Survived'].astype(int)

In [160]:
classifier = RandomForestClassifier(n_estimators=100)
classifier.fit(X_train, Y_train)
y_pred = classifier.predict(X_test)

In [161]:
result = pd.DataFrame({
    'PassengerId': PassengerId,
    'Survived': y_pred
})

In [165]:
# result.to_csv('result.csv', index=False)

In [163]:
len(result)

418